In [31]:
# Import
from haystack.database.elasticsearch import ElasticsearchDocumentStore
from backend_app.qas_core.got_data_loader import GOTDataLoader
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [32]:
# Initialize
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
data_loader = GOTDataLoader()

08/23/2020 23:45:42 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:400 request:0.015s]


In [33]:
# load data
if not data_loader.data_is_loaded():
    dicts = data_loader.load_data()
    document_store.write_documents(dicts)

In [34]:
# initialize retriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [35]:
# initialize reader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=False)

08/23/2020 23:45:47 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
08/23/2020 23:45:47 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
08/23/2020 23:45:53 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
08/23/2020 23:46:00 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
08/23/2020 23:46:03 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
08/23/2020 23:46:03 - INFO - farm.infer -   Got ya 7 parallel workers to do inference ...
08/23/2020 23:46:03 - INFO - farm.infer -    0    0    0    0    

In [36]:
# initialize finder
finder = Finder(reader, retriever)

In [37]:
# predict
prediction = finder.get_answers(question="Who is the red witch?", top_k_retriever=20, top_k_reader=10)

08/23/2020 23:46:04 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.076s]
08/23/2020 23:46:04 - INFO - haystack.retriever.sparse -   Got 20 candidates from retriever
08/23/2020 23:46:04 - INFO - haystack.finder -   Reader is looking for detailed answer in 37788 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.35 Batches/s]


In [38]:
# print answers
print_answers(prediction, details="all")

{   'answers': [   {   'answer': 'Melisandre',
                       'context': 'er Jon Snow would remain dead or return to '
                                  'life. Fans speculated that Melisandre would '
                                  'be the one to bring back Snow to life, '
                                  'after and before the tit',
                       'document_id': '-ixA4nMBreXvbNGuGruj',
                       'meta': {'name': '69_The_Red_Woman.txt'},
                       'offset_end': 80,
                       'offset_end_in_doc': 1063,
                       'offset_start': 70,
                       'offset_start_in_doc': 1053,
                       'probability': 0.5248341338350121,
                       'score': 0.795346736907959},
                   {   'answer': 'Melisandre',
                       'context': 'er Jon Snow would remain dead or return to '
                                  'life. Fans speculated that Melisandre would '
                       

In [39]:
prediction




{'question': 'Who is the red witch?',
 'no_ans_gap': -9.534895420074463,
 'answers': [{'answer': 'Melisandre',
   'score': 0.795346736907959,
   'probability': 0.5248341338350121,
   'context': 'er Jon Snow would remain dead or return to life. Fans speculated that Melisandre would be the one to bring back Snow to life, after and before the tit',
   'offset_start': 70,
   'offset_end': 80,
   'offset_start_in_doc': 1053,
   'offset_end_in_doc': 1063,
   'document_id': '-ixA4nMBreXvbNGuGruj',
   'meta': {'name': '69_The_Red_Woman.txt'}},
  {'answer': 'Melisandre',
   'score': 0.795346736907959,
   'probability': 0.5248341338350121,
   'context': 'er Jon Snow would remain dead or return to life. Fans speculated that Melisandre would be the one to bring back Snow to life, after and before the tit',
   'offset_start': 70,
   'offset_end': 80,
   'offset_start_in_doc': 1053,
   'offset_end_in_doc': 1063,
   'document_id': '9Sxw4nMBreXvbNGulcZY',
   'meta': {'name': '69_The_Red_Woman.txt'}},
